Visdom example

In [46]:
#visdom
import torch
import torch.nn as nn

import torchvision
import torchvision.datasets as dsets

Import visdom

In [47]:
import visdom
vis = visdom.Visdom()

Setting up a new session...


Text

In [48]:
vis.text("Hello, world!", env="main")

'window_3c255c1ab116ca'

Image

In [49]:
a=torch.randn(3, 200, 200)
vis.image(a)

'window_3c255c1ab95bb0'

Images

In [50]:
vis.images(torch.Tensor(3, 3, 28, 28))

c:\Users\jys74\anaconda3\envs\PyTorch\Lib\site-packages\visdom\__init__.py:1402: RuntimeWarning: invalid value encountered in cast
  grid[:, h_start:h_end, w_start:w_end] = tensor[k]


'window_3c255c1abc4210'

example (using MNIST and CIFAR10)

In [51]:
MNIST = dsets.MNIST(root="./MNIST_data",train = True,transform=torchvision.transforms.ToTensor(), download=True)
cifar10 = dsets.CIFAR10(root="./cifar10",train = True, transform=torchvision.transforms.ToTensor(),download=True)

Files already downloaded and verified


CIFAR10

In [52]:
data = cifar10.__getitem__(0)
print(data[0].shape)
vis.images(data[0], env="main")

torch.Size([3, 32, 32])


'window_3c255c1ba970c0'

MNIST

In [53]:
data = MNIST.__getitem__(0)
print(data[0].shape)
vis.images(data[0], env="main")

torch.Size([1, 28, 28])


'window_3c255c1bae21b4'

Check dataset

In [54]:
data_loader = torch.utils.data.DataLoader(dataset=MNIST,
                                          batch_size = 32,
                                          shuffle=False)

In [55]:
for num, value in enumerate(data_loader):
    value = value[0]
    print(value.shape)
    vis.images(value)
    break

torch.Size([32, 1, 28, 28])


In [56]:
vis.close(env="main")

''

Line Plot

In [57]:
Y_data = torch.randn(5)
plt = vis.line(Y=Y_data)

In [58]:
X_data = torch.Tensor([1, 2, 3, 4, 5])
plt = vis.line(Y=Y_data, X=X_data)

Line update

In [60]:
Y_append = torch.randn(1)
X_append = torch.Tensor([6])

vis.line(Y=Y_append, X=X_append, win=plt, update='append')

'window_3c255c1bcf61be'

multiple Line on sigle windows

In [64]:
num = torch.Tensor(list(range(0, 10)))
num = num.view(-1, 1)
num = torch.cat((num, num), dim=1)

plt = vis.line(Y=torch.randn(10,2), X=num)

Line info

In [65]:
plt = vis.line(Y=Y_data, X=X_data, opts = dict(title='Test', showlegend = True))

In [66]:
plt = vis.line(Y=Y_data, X=X_data, opts = dict(title='Test', legend = ['1번'], showlegend = True))

In [67]:
plt = vis.line(Y=torch.randn(10,2), X=num, opts = dict(title='Test', legend = ['1번', '2번'], showlegend = True))

make function for update line

In [68]:
def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value, are Tensor'''
    vis.line(X=num,
             Y=loss_value,
             win=loss_plot,
             update='append'
             )

In [69]:
plt = vis.line(Y=torch.Tensor(1).zero_())

for i in range(500):
    loss = torch.randn(1) + i
    loss_tracker(plt, loss, torch.Tensor([i]))

close the window

In [74]:
vis.close(env="main")

''

In [75]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

import visdom

In [77]:
import visdom

vis = visdom.Visdom()
vis.close(env="main")

Setting up a new session...


''

define loss_tracker

In [78]:
def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value, are Tensor'''
    vis.line(X=num,
             Y=loss_value,
             win=loss_plot,
             update='append'
             )

In [79]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [80]:
#parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 32

In [81]:
#MNIST dataset

mnist_train = dsets.MNIST(root='MNIST_data/',
                         train = True,
                         transform=transforms.ToTensor(),
                         download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                        train=False,
                        transform = transforms.ToTensor(),
                        download=True)

In [82]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size = batch_size,
                                          shuffle =True,
                                          drop_last=True)

In [85]:
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)

        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    

In [86]:
model = CNN().to(device)

value = (torch.Tensor(1, 1, 28, 28)).to(device)
print((model(value)).shape)

torch.Size([1, 10])


In [87]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

make plot

In [88]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(), opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))

train with loss_tracker

In [89]:
#training
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)

        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
print('Learning Finished!')    

[Epoch:1] cost = 0.12274324148893356
[Epoch:2] cost = 0.04052262380719185
[Epoch:3] cost = 0.029146932065486908
[Epoch:4] cost = 0.022723983973264694
[Epoch:5] cost = 0.018246153369545937
[Epoch:6] cost = 0.013987190090119839
[Epoch:7] cost = 0.012266572564840317
[Epoch:8] cost = 0.01126303430646658
[Epoch:9] cost = 0.009985930286347866
[Epoch:10] cost = 0.008285345509648323
[Epoch:11] cost = 0.008663928136229515
[Epoch:12] cost = 0.007057876791805029
[Epoch:13] cost = 0.006806199438869953
[Epoch:14] cost = 0.006737933028489351
[Epoch:15] cost = 0.006173445377498865
Learning Finished!


In [91]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9695000052452087
